<a href="https://colab.research.google.com/github/PrestoshTJ/ML-Projects/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O messi.txt https://raw.githubusercontent.com/JPM2002/Tech-Team---Nittany-Ai/refs/heads/main/Fall%202024/RAG/messi.txt
import openai
from openai import OpenAI
import numpy as np

--2024-12-02 23:24:41--  https://raw.githubusercontent.com/JPM2002/Tech-Team---Nittany-Ai/refs/heads/main/Fall%202024/RAG/messi.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5929 (5.8K) [text/plain]
Saving to: ‘messi.txt’

messi.txt           100%[===================>]   5.79K  --.-KB/s    in 0s      

2024-12-02 23:24:41 (52.6 MB/s) - ‘messi.txt’ saved [5929/5929]



In [6]:
client = OpenAI(api_key="sk-proj-API")

In [14]:
def chunk_text(text):
  chunkList = []
  words = text.split()
  sect = ""
  for word in words:
    if len(sect) + len(word) + 1 <= 100:
      sect += " " + word
    else:
      chunkList.append(sect)
      sect = word
  chunkList.append(sect)
  return chunkList


txt_to_embedding = {}

with open("messi.txt", "r") as file:
  contents = file.read()
  chunkList = chunk_text(contents)

  size = len(contents)
  seen = cur = 0

  for chunk in chunkList:
    seen += 1
    cur += len(chunk)
    if seen % 10 == 0:
      print(cur / size)
    response = client.embeddings.create(
        input=chunk,
        model="text-embedding-3-small"
    )
    txt_to_embedding[chunk] = response.data[0].embedding

print(txt_to_embedding)

0.16449483838212894
0.3288204433914368
0.49331528177356576
0.6591639871382636
0.8245049923844983
0.985953630055847
{' Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987),': [0.004292475059628487, -0.04614410921931267, -0.017217807471752167, -0.005523687228560448, -0.0032001936342567205, 0.02443261630833149, 0.04645071178674698, 0.005035034846514463, -0.04376791790127754, 0.008465182967483997, -0.06695494800806046, 0.01784059964120388, -0.009576627053320408, -0.04671899229288101, 0.020849164575338364, 0.058638278394937515, -0.06772146373987198, -0.01419966109097004, -0.027345366775989532, -0.04886523261666298, 0.03924548625946045, -0.012513332068920135, 0.013797241263091564, 0.013241519220173359, 0.044266149401664734, 0.02228637784719467, -0.029338302090764046, 0.00946164969354868, -0.021424049511551857, 0.03102463111281395, 0.05913650989532471, -0.024509266018867493, 0.016365062445402145, -0.0028648439329117537, 0.026578852906823158, 0.0005

In [15]:
question = input()
response = client.embeddings.create(
    input=question,
    model="text-embedding-3-small"
)
questionEmbed = response.data[0].embedding

Where was Messi born?


In [16]:
query = "Who is the greatest basketball player of all time?"

resource1 = "Lebron james is likely the greatest NBA player of all time."
resource2 = "Messi is likely the greatest soccer player of all time"
resource3 = "RAG stands for Retrieval Augmented Generation"

query_embedding = client.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    ).data[0].embedding

resource_responses = client.embeddings.create(
            input=[resource1, resource2, resource3],
            model="text-embedding-3-small"
    ).data

resource1_embedding = resource_responses[0].embedding
resource2_embedding = resource_responses[1].embedding
resource3_embedding = resource_responses[2].embedding

query = np.array(query_embedding)

resource1_embedding = np.array(resource1_embedding)
resource2_embedding = np.array(resource2_embedding)
resource3_embedding = np.array(resource3_embedding)

def cosine_simularity(A, B):
    return np.dot(A,B) / ( np.linalg.norm(A) * np.linalg.norm(B) )

simularity1 = cosine_simularity(query_embedding, resource1_embedding)
simularity2 = cosine_simularity(query_embedding, resource2_embedding)
simularity3 = cosine_simularity(query_embedding, resource3_embedding)

print(f"Simularity of query and resource1: {simularity1}")
print(f"Simularity of query and resource2: {simularity2}")
print(f"Simularity of query and resource3: {simularity3}")

Simularity of query and resource1: 0.7120352025139127
Simularity of query and resource2: 0.5607410563606989
Simularity of query and resource3: -0.016951191459983996


In [17]:
cosSims = []
for chunk in txt_to_embedding:
  cosSims.append([cosine_simularity(questionEmbed, txt_to_embedding[chunk]), chunk])
cosSims.sort(reverse=True)
print(cosSims)

[[0.6543769824407542, 'billion in career earnings. Early life Messi was born on 24 June 1987 in Rosario, Santa Fe'], [0.6146778829794577, 'Argentina, Messi relocated to Spain to join Barcelona at age 13, and made his competitive debut at'], [0.5993659707308496, 'playmaker, Messi has scored over 850 senior career goals for club and country. Born in Rosario,'], [0.5896266620031057, ' Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987),'], [0.5735199288971523, 'Province,[21] the third of four children of Jorge Messi, a steel factory manager, and his wife Celia'], [0.5452573863077356, 'Olympics. After his senior debut in 2005, Messi became the youngest Argentine to play and score in a'], [0.5361799116620859, 'treble in Spanish football; that year, aged 22, Messi won the first of his four consecutive Ballons'], [0.5277197865930139, "the club by his second season. An Argentine international, Messi is the national team's all-time"], [0.49513691181

In [19]:
top_5 = []
for i in range(0,4):
  top_5.append(cosSims[i][1])
print(top_5)

['billion in career earnings. Early life Messi was born on 24 June 1987 in Rosario, Santa Fe', 'Argentina, Messi relocated to Spain to join Barcelona at age 13, and made his competitive debut at', 'playmaker, Messi has scored over 850 senior career goals for club and country. Born in Rosario,', ' Lionel Andrés Messi[note 1] (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ⓘ; born 24 June 1987),']


In [22]:
message = question + " " + " ".join(top_5)
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": message}
    ]
)
print(question)
print(response.choices[0].message.content)

Where was Messi born?
Lionel Messi was born in Rosario, Santa Fe, Argentina.
